In [ ]:
from resources.workspace import *
from IPython.display import display
from scipy.integrate import odeint
import copy

%matplotlib inline

# Lyapunov exponents and eigenvalues

A **Lypunov exponent** can be understood loosely as a kind of generalized eigenvalue for time-depenent linear transformations, or for the linearization of a nonlinear evolution.

What do eigenvalues tell us about a matrix and why might the above results seem intuitive? 

Consider the equation for the <em>evolution</em> of the pertubations <span style='font-size:1.25em'>$\boldsymbol{\delta}^i_k$</span></a>.  We can write,
<h3>
$$\begin{align}
& \boldsymbol{\delta}_k^i = \mathbf{x}_k^c - \mathbf{x}_k^i \\
\Rightarrow & \dot{\boldsymbol{\delta}}_k^i = f(\mathbf{x}_k^c) - f(\mathbf{x}_k^i).
\end{align}$$
</h3>
But for small perturbations, we can reasonably make an approximation with a Taylor expansion,
<h3>
$$\begin{align}
 f(\mathbf{x}_k^c) - f(\mathbf{x}_k^i) \approx \nabla f\rvert_{\mathbf{x}c}  \boldsymbol{\delta}^i_k, & &   
\end{align}$$
</h3> 
where the term, 
<h2>
$$f\rvert_{\mathbf{x}c}$$
</h2>
is the gradient with respect to the state variables, i.e., the **[Jacobian matrix](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant)**, evaluated at the control trajectory.

This means that for small perturbations, the evolution is well approximated by the linear Jacobian equations, and we can think of these linear equations having some kind of generalized eigenvalues, describing the invariant (exponential) growth and decay rates for the system. 

#### The power method

The method of breeding errors above is conceptually very similar to the classical [power method](https://en.wikipedia.org/wiki/Power_iteration) for finding the leading eigenvalue of a diagonalizable matrix:

* Suppose <span style='font-size:1.25em'>$\mathbf{M}\in\mathbb{R}^{n\times n}$</span> is a diagonalizable matrix, with eigenvalues,
<h3>
$$
\rvert \mu_1 \rvert > \rvert\mu_2\rvert \geq \cdots \geq \rvert\mu_n\rvert,
$$
</h3>
i.e., <span style='font-size:1.25em'>$\mathbf{M}$</span> has a single eigenvalue of magnitude greather than all its others.   

* Let <span style='font-size:1.25em'>$\mathbf{v}_0 \in \mathbb{R}^n$</span> be a randomly selected vector, with respect to the Gaussian distribution on <span style='font-size:1.25em'>$\mathbb{R}^n$</span>. 

* We define the algorithm,
<h3>
$$\begin{align}
\mathbf{v}_{k+1} \triangleq \frac{\mathbf{M} \mathbf{v}_k}{ \left\rvert \mathbf{M} \mathbf{v}_k\right\rvert} & &
\widehat{\mu}_{k+1} \triangleq \mathbf{v}_{k+1}^{\rm T} \mathbf{M} \mathbf{v}_{k+1} 
\end{align}$$
</h3>
as the power method.  

It is easy to verify that with probability one, the sequence <span style='font-size:1.25em'>$\widehat{\mu}_k$</span> converges to the dominant eigenvalue, <span style='font-size:1.25em'>$\mu_1$</span>, and <span style='font-size:1.25em'>$\mathbf{v}_k$</span> converges to an eigenvector for the dominant eigenvalue. 

**Exc 4.20**: Fill in the code below to write an algorithm for the power method.

In [ ]:
def power_method(M, v, number_iterations):
    """takes a diagonalizable matrix M and returns approximations for the leading eigenvector/eigenvalue"""
    
    for i in range(number_iterations):
        ### fill in missing lines here

    return v, mu

In [ ]:
# Example solution

# show_answer('power_method')

**Exc 4.22**: Test your solution to **Exc 4.20**.  Use the code and slider below to study the rate of convergence.  In this case, the matrix will have eigenvalues 
<h3>$$\begin{align}
\left\{r^i : \hspace{2mm} i =0, 1, 2, \hspace{2mm} \text{and} \hspace{2mm} r\in(1,2]\right\}
\end{align}$$</h3>
The parameter <span style='font-size:1.25em'>$k$</span> defines how many iterations of the power method are computed.  How does the value <span style='font-size:1.25em'>$r$</span> affect the number of iterations necessary to reach convergence?

In [ ]:
def animate_power_convergence_rate(k=1, r=1.5):    
    
    # We define a well conditioned matrix M, depending on the ratio of the eigenvalues
    M = array([r ** i for i in range(3)])
    M = np.diag(M)
    e_3 = array([0, 0, 1])

    # define a random initial condition
    np.random.seed(0)
    v = randn(3)
    v = v / sqrt(v.T @ v)
    
    # and storage for the series of approximations
    v_hist = zeros(k+1)
    v_hist[0] = e_3.T @ v 
    
    mu_hist = zeros(k+1)
    mu_hist[0] = v.T @ M @ v
    
    # for the number of iterations k, return the power method approximation
    for it in range(1,k+1):
        np.random.seed(0)
        v, mu = power_method(M, v, it)
        v_hist[it] = np.arccos(e_3.T @ v)
        mu_hist[it] = mu
     
    # PLOTTING
    fig = plt.figure(figsize=(16,8))
    ax1 = plt.subplot(121)
    ax2 = plt.subplot(122)
    ax1.plot(range(0,k+1), v_hist)
    ax2.plot(range(0,k+1), mu_hist)
    ax1.set_ybound([0,1.05])
    ax2.set_ybound([.9,4])
    
    t_scl = np.floor_divide(k+1, 10)
    
    ax1.set_xticks(range(0, k+1, t_scl + 1))
    ax2.set_xticks(range(0, k+1, t_scl + 1))
    
    ax1.text(0, 1.07, r'Angle between $\mathbf{v}_k$ and eigenvector', size=20)
    ax2.text(0, 4.05, r'Value of $\mu_k$', size=20)
    ax1.tick_params(
        labelsize=20)
    ax2.tick_params(
        labelsize=20)
    
    plt.show()

w = interactive(animate_power_convergence_rate, k=(1,15), r=(1.05,2, .05))
w

<b>Exc 4.24.a </b>: Suppose the power method is performed on a generic diagonalizable matrix <span style='font-size:1.25em'>$\mathbf{M}\in\mathbb{R}^{n\times n}$</span>, with eigenvalues
<h3>$$\begin{align}
\rvert \mu_1 \rvert > \rvert\mu_2 \rvert\geq \cdots \geq \rvert\mu_n \rvert,
\end{align}$$</h3>
with a randomly selected initial vector <span style='font-size:1.25em'>$\mathbf{v}_0$</span>, with respect to the Gaussian distribution on <span style='font-size:1.25em'>$\mathbb{R}^n$</span>.

Can you conjecture what is the order of convegence for the sequences <span style='font-size:1.25em'>$\mathbf{v}_k$</span> and <span style='font-size:1.25em'>$\widehat{\mu}_k$</span>? 

**Hint**: the rate depends on the eigenvalues.

**Exc 4.42.b***: Prove the rate of convergence.

In [ ]:
# Answer

# show_answer('power_method_convergence_rate')

<b>Exc 4.28* </b>: We have brushed over why the algorithm described above converges with *probability one*, can you prove why this is the case?

In [ ]:
# Answer

# show_answer('probability_one')

<b>Exc 4.30.a </b>: Let <span style='font-size:1.25em'>$\widehat{\mu}_k$</span> be defined as in **Exc 4.24**.  Suppose we define a sequence of values,
<h3>$$\begin{align}
\widehat{\lambda}_T = \frac{1}{T} \sum_{k=1}^T\log\left(\rvert \widehat{\mu}_k\right \rvert).
\end{align}$$</h3>
Answer the following:
<ol>
  <li> Can you conjecture what <span style='font-size:1.25em'>$\widehat{\lambda}_T$</span> converges to as <span style='font-size:1.25em'>$T \rightarrow \infty$</span>?  
  
  **Hint**: Use the fact that <span style='font-size:1.25em'>$\widehat{\mu}_k \rightarrow \mu_1$</span> as <span style='font-size:1.25em'>$k \rightarrow \infty$</span></li>
  
  <li> Suppose we define the Lyapunov exponents as the log-average growth rates of the matrix <span style='font-size:1.25em'>$\mathbf{M}$</span>.  What can you guess about the relationship between the eigenvalues and the Lyapunov exponents of the matrix <span style='font-size:1.25em'>$\mathbf{M}$</span>?</li>
<ol>

<b>Exc 4.30.b*</b>: Prove that the limit
<h3>$$\begin{align}
\lim_{T \rightarrow \infty} \widehat{\lambda}_T
\end{align}$$</h3>
exists, and what quantity it converges to.

In [ ]:
# Answers

# show_answer('lyapunov_exp_power_method')

#### The QR algorithm

The power method is an intuitive method for finding the dominant eigenvalue for a special class of matrices.  However, we generally want to find directions that may also be growing, though more slowly than the dominant direction.  

Intuitively, if we are tracking a control trajectory with data assimilation and we corrected the forecast errors only in the direction of dominant error growth, we may still lose track of the control trajectory, only it would be more slowly than the dominant rate of growth.

There is a simple generalization of the power method for finding higher dimensional subspaces.  We may consider *separating* perturbations into directions that grow at different rates.  One easy way to perform this is to construct a *moving frame* in the span of the perturbations.  If there is only one perturbation, then the power method constructs precisely a 1-dimensional moving frame, with a vector that is always of norm 1. 

If there are two perturbations we can construct a moving frame in the span of the perturbations with a [Gram-Schmidt](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process) step.  A visualization of the Gram-Schmidt process for three vectors is picuted in the visualization below.

<div style='width:900px'>
<img src="./resources/Gram-Schmidt_orthonormalization_process.gif">
</div>

**By Lucas V. Barbosa [Public domain], <a href="https://commons.wikimedia.org/wiki/File:Gram-Schmidt_orthonormalization_process.gif">from Wikimedia Commons</a>**

In our case, suppose we have two initial, orthogonal vectors
<h3>$$
\mathbf{x}_0^1, \mathbf{x}_0^2
$$</h3>
which we will propagate forward.  We define for each $j=1,2$,
<h3>$$
\widehat{\mathbf{x}}^j_1 \triangleq \mathbf{M} \mathbf{x}^j_0.
$$</h3>
The first vector will follow the usual power method, i.e.,
<h3>$$
\mathbf{x}^1_1 \triangleq \frac{\widehat{\mathbf{x}}_1^1}{\left\rvert \widehat{\mathbf{x}}_1^1\right\rvert},
$$</h3>

However, we want to separate the second vector <span style='font-size:1.25em'>$\widehat{\mathbf{x}}_1^2$</span> so the new perturbations don't align.  We thus remove the components in the direction of <span style='font-size:1.25em'>$\mathbf{x}_1^1$</span>, before we normalize <span style='font-size:1.25em'>$\widehat{\mathbf{x}}_1^2$</span>.  
<h3>$$\begin{align}
\mathbf{y}^2_1 &\triangleq \widehat{\mathbf{x}}_1^2- \langle \mathbf{x}_1^1,  \widehat{\mathbf{x}}^2_1\rangle \mathbf{x}_1^1 \\
\mathbf{x}^2_1 & \triangleq \frac{\mathbf{y}_1^2}{\left\rvert \mathbf{y}_1^2 \right\rvert}
\end{align}$$</h3>

It is easy to see by definition that <span style='font-size:1.25em'>$\mathbf{x}_1^1, \mathbf{x}_1^2$</span> are orthogonal, but we can also show an important dynamical property with this transformation.  Define the following coefficients,
<h3>$$
\begin{align}
U^{11}_1 &=\left\rvert \widehat{\mathbf{x}}_1^1\right\rvert \\
U^{22}_1 &=\left\rvert \mathbf{y}_1^2 \right\rvert \\
U^{12}_1 &= \langle \mathbf{x}^1_1, \widehat{\mathbf{x}}_1^2\rangle 
\end{align}
$$<h3>

**Exc 4.32**: Can you write the recursion for the vectors <span style='font-size:1.25em'>$\mathbf{x}_0^1, \mathbf{x}_0^2$</span> transformed into <span style='font-size:1.25em'>$\mathbf{x}_1^1,\mathbf{x}_1^2$</span> with the coefficients <span style='font-size:1.25em'>$U^{ij}_1$</span> defined above in matrix form?  Can you write the recursion for an arbitrary number of steps $k\in\{1,2,\cdots\}$?

In [ ]:
# Answer

# show_answer('gram-schmidt')

The above procedure defines the *naive* QR algorithm --- one should note that there are more computationally efficient versions of this algorithm utilized in standard linear algebra software libraries.  However, this simple intuition forms the basis for many powerful theoretical results.  

The QR algorithm (in its refined version) is the standard method for computing the <b>[Schur decomposition](https://en.wikipedia.org/wiki/Schur_decomposition)</b> for a matrix, which is used for many purposes as it is a numerically stable alternative to the <b>[Jordan Cannonical Form](https://en.wikipedia.org/wiki/Jordan_normal_form)</b>, pictued below:

<div style='width:900px'>
<img src="./resources/Jordan_blocks.svg">
</div>

**By Jakob.scholbach [<a href="https://creativecommons.org/licenses/by-sa/3.0">CC BY-SA 3.0</a> or <a href="http://www.gnu.org/copyleft/fdl.html">GFDL</a>], <a href="https://commons.wikimedia.org/wiki/File:Jordan_blocks.svg">from Wikimedia Commons</a>**

The Jordan Canonical form is highly appealing as it is the diagonal or "almost-diagonal" form of a matrix.  However, this is highly unstable to compute in most applications.

The Schur decomposition relaxes this further, from "almost-diagonal" to upper triangular, another useful form for a matrix.  In particular, the Schur decomposition is one approach to find **all eigenvalues** for a matrix, separated into a **chain of descending growth and decay rates**.

<b>Exc 4.34</b>: Suppose a matrix <span style='font-size:1.25em'>$\mathbf{M}$</span> has a Schur decomposition, given as,
<h3> $$ \begin{align}
\mathbf{M} = \mathbf{Q} \mathbf{U} \mathbf{Q}^{\rm T},
\end{align}$$ </h3>
where <span style='font-size:1.25em'>$\mathbf{U}$</span> is strictly upper triangular, and <span style='font-size:1.25em'>$\mathbf{Q}$</span> is orthogonal such that <span style='font-size:1.25em'>$\mathbf{Q}^{\rm T} = \mathbf{Q}^{-1}$</span>.  Can you prove that the eigenvalues of <span style='font-size:1.25em'>$\mathbf{M}$</span> are the diagonal elements of <span style='font-size:1.25em'>$\mathbf{U}$?</span> 

If <span style='font-size:1.25em'>$\mathbf{Q}^j$</span> is the $j$-th column of <span style='font-size:1.25em'>$\mathbf{Q}^j$</span>, what does the product
<h3>$$\begin{align}
\left(\mathbf{Q}^j\right)^{\rm T} \mathbf{M} \mathbf{Q}^j
\end{align}$$</h3>
equal in terms of the earlier quantities? <b>Hint</b>: how does this relate to the power method?

In [ ]:
# Answer

# show_answer('schur_decomposition')

<b>Exc 4.36</b>: Can you conjecture what the Schur decomposition will take in the case that the matrix <span style='font-size:1.25em'>$\mathbf{M}$</span> has complex eigenvalues?

In [ ]:
# Answer

# show_answer('real_schur')

### Next: [Lyapunov vectors and ensemble based covariances](T4 - Lyapunov vectors and ensemble based covariances.ipynb)